# Classification

K. Leighly 2017

This lecture was drawn from the following sources:
 - Ivezic chapter 9
 - Bishop Chapter 4 & 5
 - "Machine Learning An Algorithmic Perspective, 2nd Edition" by Stephen Marsland, Chapter 3 & 4


##  Outline and Motivation

Classification and regression go hand in hand. Regression seeks a continuous function between the input data and the target variables, while classification seeks to classify the input data into two or more bins called decision regions that are divided by decision boundaries or decision surfaces.

We will begin classification referring to Chapter 4 of Bishop in order to build a framework onto which we can build various special topics. In this chapter, we will consider principally linear models of classification, where the decision surfaces are a linear function of the input variable $x$ and are therefore defined by $(D-1)$-dimensional hyperplane within the $D$-dimensional input space.  Within this section, we will also discuss material from Ivezic Chapter 9.1-9.3, and scikit-learn Classification 1.2.

We will then move on to more complicated, multi-layer classification scenarios, including neural net.  For this, we will use Bishop Chapter 5, and 

## Linear Models for Classification - Basics

First, let's establish some notation. It is convenient to use a 1-of-$K$ coding scheme which simply means that the target will be a vector $\mathbf{t}$ of length $K$ where all elements $t_k$ of $\mathbf{t}$ will be zero except the one corresponding to class $k$. Note that this is a handy way to convert a label into a numerical representation.

For example, if $K=5$ then a target from Class 2 would have:

$$\mathbf{t} = (0,1,0,0,0)^T.$$

It is noteworthy, however, that the classification programs from scikit-learn that I have played around with do not use one-of-K coding.  Their target values are contained in a single vector.  See examples below.

More notation. Recall that for the linear regression models, the model prediction $y(\mathbf{x},\mathbf{w})$ was given by a linear function of parameters $\mathbf{w}$. In the simplest form, where the function is also linear in the input variables, the functional form was $y(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + w_0$.

For classification, one needs to predict class labels, or more generally, probabilities associated with each class label. So we make a generalization of the model whereby the linear function of $\mathbf{w}$ is transformed using the nonlinear function $f$ so that

$$y(\mathbf{x}) = f(\mathbf{w}^T \mathbf{x} + w_0).$$

This function is alternatively called an **activation function**, and its purpose is to map $\mathbf{w}^T\mathbf{x} + w_0$ (or whatever) to various classes.  The decision surfaces will be given by $y(\mathbf{x}) = \text{constant}$, so that $\mathbf{w}^T \mathbf{x} + w_0 = \text{constant}$, and so the decision surfaces are linear. 

This class of models is called _generalized linear models_, even though they are not linear in the parameters due to the presence of $f$. But they still form a simpler case compared with the nonlinear models.



### Discriminant Functions

A discriminant function is one that takes an input vector $\mathbf{x}$ and assigns it to one of the $K$ classes, denoted $\mathcal{C}_K$. As noted above, first we consider linear discriminants, so the surfaces are hyperplanes.

First consider two classes. Let's take

$$y(\mathbf{x}) = \mathbf{w}^T \mathbf{x} + w_0$$

where $\mathbf{w}$ is called the _weight vector_ and $w_0$ is the _bias_. (The negative of the bias can be called the _threshold_). As will be discussed below, the bias can be simply thought of as an offset.

An object will be assigned to class $\mathcal{C}_1$ if $y(\mathbf{x}) \ge 0$ and to $\mathcal{C}_2$ otherwise. The _decision boundary_ (red in the sketch below) is defined by $y(\mathbf{x}) = 0$, which corresponds to a $D-1$ dimensional hyperplane within the $D$ dimensional space.

Let's look at the geometry of this hypersurface. Consider two points, $x_A$ and $x_B$ that lie on the _decision surface_ (red), and therefore $y(\mathbf{x}_A)-y(\mathbf{x}_B)=y(\mathbf{x}_A) = y(\mathbf{x}_B) = 0$.  Therefore, $\mathbf{w}^T (\mathbf{x}_A - \mathbf{x}_B) = 0$. Therefore, $\mathbf{w}$ is orthogonal to every vector lying on the decision surface, and so $\mathbf{w}$ determines the _orientation_ of the decision surface.  The sketch below illustrates Bishop Figure 4.1.

![hand-drawn sketch of Bishop 4.1](http://www.nhn.ou.edu/~leighly/bishop_4.1_sketch.tiff)

Moreover, if $\mathbf{x}$ is a point on the surface, then $y(\mathbf{x}) = 0$, so the distance from the origin to the decision surface is:

$$\frac{\mathbf{w}^T \mathbf{x}}{\lVert \mathbf{w} \rVert} = \frac{w_0}{\lVert \mathbf{w} \rVert}.$$

Thus, the bias parameter $w_0$ determines the location of the decision surface. Moreover, the value of $y(\mathbf{x})$ gives a signed measure of the perpendicular distance $r$ of the point $\mathbf{x}$ from the decision surface. 

In this case, the decision surfaces are $D$-dimensional hyperplanes passing through the origin of the $D+1$-dimensional expanded input space.




## Multiple Classes

As seen above, the case for two classes is simple enough.  So, it is tempting to generate a $K$-class discriminant by combining a number of two-class discriminant functions. The problem, illustrated in the hand-drawn sketch of Bishop Figure 4.2 below is that it is not always all that well defined.

![bishop 4.2 sketch](http://www.nhn.ou.edu/~leighly/bishop_4.2_sketch.tiff)


The solution to the ambiguities is to consider a single $K$-class discriminant comprising $K$ linear functions of the form

$$y_k(\mathbf{x}) = \mathbf{w}^T_K \mathbf{x} + w_0.$$

Then, assign a point $\mathbf{x}$ to class $\mathcal{C}_k$ if $y_k(\mathbf{x}) > y_j(\mathbf{x})$ for all $j \neq k$. The decision boundary between class $\mathcal{C}_k$ and class $\mathcal{C}_j$ is given by $y_k(\mathbf{x}) = y_j(\mathbf{x})$ and therefore corresponds to a $(D-1)$-dimensional hyperplane defined by

$$(\mathbf{w}_k - \mathbf{w}_j)^T \mathbf{x} = (w_{k0}-w_{j0}) = 0.$$

This looks just like the decision boundary for the two-class case.  The sketch for this situation, adopted from Bishop Figure 4.3, is seen below.

![Bishop 4.3 sketch](http://www.nhn.ou.edu/~leighly/bishop_4.3_sketch.tiff)


## Least Squares for Classification

Next we need to determine the $\mathbf{w}$, i.e., the parameters describing the decision boundary.  We will start by using a least squares approach, since that worked for linear regression.  We will find that it generally doesn't work that well, due at least in part to the assumption, when using least squares that the data are distributed normally. So if there are significant outliers, it doesn't work.

Consider a classification problem with $K$ classes, with a 1-of-$K$ binary coding scheme for target vector $\mathbf{t}$.

Each class is described by

$$y_k(\mathbf{x}) = \mathbf{w}_k^T \mathbf{x} + w_{k0}$$

where $k = 1,\ldots,K$. These can be grouped together with

$$\mathbf{y}(\mathbf{x}) = \widetilde{\mathbf{W}}^T \widetilde{\mathbf{x}}$$

where $\widetilde{\mathbf{W}}$ is a matrix whose $k$th column comprises the $D+1$-dimensional vector $\widetilde{\mathbf{w}}= (w_{k0}, \mathbf{w}^T)^T$ and $\widetilde{\mathbf{x}}$ is the augmented input vector $(1,\mathbf{x}^T)^T$, as defined above.

So a new input $\mathbf{x}$ is assigned to a class for which the output $y_k = \widetilde{\mathbf{w}}_k^T \widetilde{\mathbf{x}}$ is larger.

We determine the parameter matrix $\widetilde{\mathbf{W}}$ by minimizing the sum-of-squares error function. 

Consider the training set $\{\mathbf{x}_n, \mathbf{t}_n\}$ where $n=1,\ldots,N$, and define a matrix $\mathbf{T}$ where the $n$th row is the vector $\mathbf{t}^T_n$, together with a matrix $\widetilde{\mathbf{X}}$ whose $n$th row is $\widetilde{\mathbf{x}}^T_n$. The sum of squares error function then is:

$$E_D(\widetilde{\mathbf{W}}) = \frac{1}{2} Tr \left \{ (\widetilde{\mathbf{X}} \widetilde{\mathbf{W}} - \mathbf{T})^T (\widetilde{\mathbf{X}} \widetilde{\mathbf{W}} -\mathbf{T}) \right \}.$$

I believe that the trace is used because target vector has the 1-of-$K$ coding scheme, but I didn't check.

As usual, set the derivative with respect to $\widetilde{\mathbf{W}}$ equal to zero yields:

$$\widetilde{\mathbf{W}} = (\widetilde{\mathbf{X}}^T \widetilde{\mathbf{X}})^{-1} \widetilde{\mathbf{X}}^T \mathbf{T} = \widetilde{\mathbf{X}}^\dagger \mathbf{T}$$

where recall that the $\widetilde{\mathbf{X}}^\dagger$ is the [pseudo-inverse](https://en.wikipedia.org/wiki/Moore–Penrose_inverse), which was briefly discussed in the regression lecture when we discussed the design matrix (i.e., $\boldsymbol{\Phi}^\dagger \equiv (\boldsymbol{\Phi}^T \boldsymbol{\Phi})^{-1} \boldsymbol{\Phi}^T.$

Then

$$\mathbf{y}(\mathbf{x}) = \widetilde{\mathbf{W}}^T \widetilde{\mathbf{x}} = \mathbf{T}^T (\widetilde{\mathbf{X}}^\dagger)^T \widetilde{\mathbf{x}}.$$

Thus we have the parameters defining the decision regions in an exact closed form. However, this method does not always give good results; specifically, it performs badly when there are outliers. This is because least squares corresponds to maximum likelihood under the assumption of a Gaussian distribution, which we clearly don't always have.



## Probabilistic Generative Models

There are basically two ways to approach classification.  
1. Generative classification - where a full model for the probability density for each class will be developed.
2. Discriminative classification - where the decision boundary that separates classes is found.  This is said to be better in high dimensional problems.

In this section, we will consider generative classification.  We will model the class-conditional densities $p(\mathbf{x}|\mathcal{C}_k)$, as well as the class priors $p(\mathcal{C}_k)$, and obtain the posterior probability $p(\mathcal{C}_k| \mathbf{x})$ through Bayes' theorem.



Consider the case of two classes. The posterior probability of class $\mathcal{C}_1$ is, via Bayes Rule:

$$p(\mathcal{C}_1 | \mathbf{x}) = \frac{p(\mathbf{x} | \mathcal{C}_1) p(\mathcal{C}_1)} {p(\mathbf{x} | \mathcal{C}_1) p(\mathcal{C}_1) + p(\mathbf{x}|\mathcal{C}_2) p(\mathcal{C}_2)}$$

$$ = \frac{1}{1+\exp (-a)} = \sigma (a)$$

where we have defined

$$a= \ln \frac{p(\mathbf{x} | \mathcal{C}_1) p(\mathcal{C}_1)} {p(\mathbf{x} | \mathcal{C}_2) p(\mathcal{C}_2)} $$

and $\sigma(a)$ is the _logistic sigmoid function_ defined by

$$\sigma(a) = \frac{1}{1+\exp(-a)}$$

and shown below.

![wikipedia commons](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/600px-Logistic-curve.svg.png)


The logistic sigmoid function is an important one in classification as an example of an activation function.  It is a smooth curve that asymptotically goes to zero or one, so it fulfills the requirements of an activation function.  Note that it arose naturally from Bayes' rule.

Some facts about the logistic sigmoid function:

- Sigmoid means S-shaped.
- it is a symmetric function, i.e., $\sigma(-a) = 1-\sigma(a)$.
- The inverse of the logistic sigmoid is given by $$a= \ln \left ( \frac{a}{1-a} \right )$$ which is known as the _logit fuction_.
- $a$ represents the log of the ratio of probabilities $\ln [p(\mathcal{C}_1 | \mathbf{x}) / p(\mathcal{C}_2 | \mathbf{x})]$, and is known as the _log odds_.


For $K > 2$ classes, this generalizes to

$$p(\mathcal{C}_k | \mathbf{x}) = \frac{p(\mathbf{x} | \mathcal{C}_k) p(\mathcal{C}_k)} {\sum_j p(\mathbf{x} | \mathcal{C}_j) p(\mathcal{C}_j)}$$
$$ = \frac{\exp (a_k)}{\sum_j \exp(a_j)}.$$

This is known as the normalized exponential, and is it a multi class generalization of the logistic sigmoid. Here, $a_k$ are defined as

$$a_k = \ln (p(\mathbf{x} | \mathcal{C}_k) p(\mathcal{C}_k)).$$

The normalized exponential is also known as the [_softmax function_](https://en.wikipedia.org/wiki/Softmax_function) and it will be refered to by that name henceforth.



### Continuous Inputs

Now let's see what we can do with this formalism. To start with, let's have the likelihood be a Gaussian function.

$$p(\mathbf{x} | \mathcal{C}_k) = \frac{1}{(2 \pi)^{D/2}}
\frac{1}{|\boldsymbol{\Sigma}|^{1/2}} \exp \left \{
-\frac{1}{2}(\mathbf{x} - \boldsymbol{\mu}_k)^T
\boldsymbol{\Sigma}^{-1} (\mathbf{x} - \boldsymbol{\mu}_k) \right \}.$$

Consider two classes. From above, we can write $p(\mathcal{C}_1 | \mathbf{x})$ as

$$p(\mathcal{C}_1 | \mathbf{x}) = \sigma(\mathbf{w}^T \mathbf{x} + w_0)$$

where we define 

$$\mathbf{w} = \boldsymbol{\Sigma}^{-1} (\boldsymbol{\mu}_1 - \boldsymbol{\mu}_2)$$

and

$$w_0 = -\frac{1}{2} \boldsymbol{\mu}_1^T \boldsymbol{\Sigma}^{-1} \boldsymbol{\mu}_1 + \frac{1}{2} \boldsymbol{\mu}_2^T \boldsymbol{\Sigma}^{-1} \boldsymbol{\mu}_2 + \ln \frac{p(\mathcal{C}_1)} {p(\mathcal{C}_2)}.$$

Here, $\sigma$ is the logistic sigmoid, as defined above.
This is just derived from the equations above, plus algebra. One thing to note is that the quadratic terms in $\mathbf{x}$ from the exponents in the Gaussian densities have cancelled due to the assumption of _common covariance matrices for both classes_, which may not be true in general. 

The result, for a two-dimensional $\mathbf{x}$, in probability space, would be two two-dimensional Gaussian distributions that intersect along the decision boundary, where the probability for $p(\mathcal{C}_1 |\mathbf{x}$ would be a continous function between 0 and 1.  That is, for any value of $\mathbf{x}$, there will be a probability that the object is in $\mathcal{C}_1$ and a probability that it is in $\mathcal{C}_2$.  The decision boundary is just where the probabilities are equal.

So this is a kind of a mapping, basically from likelihood space $p(\mathbf{x} | \mathcal{C}_k)$, where the relevant thing, in a sense, is the distribution of the data $\mathbf{x}$, to posterior space $p(\mathcal{C}_1 | \mathbf{x})$, where the relevant thing is the class $\mathcal{C}_k$, all done handily by writing in terms of the logistic sigmoid function.

The decision boundary correspond to surfaces along which $p(\mathcal{C}_k | \mathbf{x})$ are constant, and, since $p(\mathcal{C}_1 | \mathbf{x})$ is a linear function of $\mathbf{x}$, will themselves be linear functions of $\mathbf{x}$. Also note that the priors appear only in $w_0$, and so they have the effect of making parallel shifts in the decision boundary.

The decision boundary between two one-dimensional Gaussian distributions is illustrated in Figure 9.1 from Ivezic below:

![Figure 9.1, Ivezic](http://www.astroml.org/_images/fig_bayes_DB_1.png)

For the case of two 2-dimensional Gaussians, see Bishop Figure 4.10.

This can be generalized to $K$ classes as:

$$a_k(\mathbf{w}) = \mathbf{w}^T_k \mathbf{w} + w_{k0}$$

where

$$w_k = \boldsymbol{\Sigma}^{-1} \boldsymbol{\mu}_k$$

and

$$w_{k0} = -\frac{1}{2} \boldsymbol{\mu}_k^T \boldsymbol{\Sigma}^{-1} \boldsymbol{\mu}_k + \ln p(\mathcal{C}_k).$$

This yields $a_k(\mathbf{x})$ that are again linear functions of $\mathbf{x}$ due to the shared covariances as above. The resulting decision boundaries will be found where the two largest posterior probabilities (in that area) are equal, and they will again be linear functions of $\mathbf{x}$.

If we relax the assumption of a shared covariance matrix and therefore allow each class-conditioned density $p(\mathbf{x} | \mathcal{C}_k)$ to have its own covariance matrix $\boldsymbol{\Sigma}_k$, then quadratic functions of $\mathbf{x}$ will occur, yielding a quadratic discriminant. In that case, decision boundaries will be parabolas.

## Maximum Likelihood Solution

Now we have the class-conditioned densities $p(\mathbf{x} | \mathcal{C}_k)$, then, including the priors, we can determine the values of the parameters using maximum likelihood.

First consider just two classes. The data set comprises $\mathbf{x}$ and the corresponding class labels, with the one-in-k encoding, rather than a target variable, as with regression. 

The data set is $\{\mathbf{x}_n,t_n \}$, where $n=1,\ldots,N$, and $t_n=1$ denotes class $\mathcal{C}_1$, and $t_n = 0$ denotes class $\mathcal{C}_2$. 

Let the prior class probability $p(\mathcal{C}_1) = \pi$, so that $p(\mathcal{C}_2) = 1-\pi$. For an individual data point $\mathbf{x}_n$ from class $\mathcal{C}_1$, we have $t_n=1$, and so:

$$p(\mathbf{x}_n,\mathcal{C}_1) = p(\mathcal{C}_1) p(\mathbf{x}_n | \mathcal{C}_1) =\pi \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_1,\boldsymbol{\Sigma}).$$

Class 2 is similar:

$$p(\mathbf{x}_n,\mathcal{C}_2) = p(\mathcal{C}_2) p(\mathbf{x}_n | \mathcal{C}_2) =(1-\pi) \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_2,\boldsymbol{\Sigma}).$$

And the likelihood function is therefore

$$p(\mathbf{t},\mathbf{X} | \pi, \boldsymbol{\mu}_1,\boldsymbol{\mu}_2,\boldsymbol{\Sigma}) = \prod_{n=1}^N [\pi \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_1,\boldsymbol{\Sigma})]^{t_n} [(1-\pi) \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_2,\boldsymbol{\Sigma})]^{1-t_n},$$

where $\mathbf{t} = (t_1,\ldots,t_N)^T$.

This all looks really familiar, so we know that the next thing we need to do is take the log and maximize with respect to various parameters. 

Consider first maximizing with respect to $\pi$. The terms in the likelihood function (i.e., log of the probability density) that depend on $\pi$ are

$$\sum_{n=1}^N \{t_n \ln \pi + (1-t_n) \ln (1-\pi) \}.$$

Setting the derivative with respect to $\pi$ equal to zero yields

$$\pi = \frac{1}{N} \sum_{n=1}^N t_n = \frac{N_1}{N} = \frac{N_1}{N_1+N_2}$$

where $N_1$ is the total number of data points in $\mathcal{C}_1$ and $N_2$ is the total number of data points in $\mathcal{C}_2$. 

Thus, the maximum likeihood estimate for $\pi$ is simply the fraction of points in $\mathcal{C}_1$. Moreover, the result is easily generalized to the $K$ class situation to be the fraction of points in each class.

Now consider the maximization with respect to $\boldsymbol{\mu}_1$. The factors in the log likelihood function that depend on $\boldsymbol{\mu}_1$ are as follows:

$$\sum_{n=1}^N t_n \ln \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu_1}, \boldsymbol{\Sigma}) = -\frac{1}{2} \sum_{n=1}^N t_n (\mathbf{x}_n - \boldsymbol{\mu}_1)^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}_n - \boldsymbol{\mu}_1) + const.$$

Setting the derivative with respect to $\boldsymbol{\mu}_1$ equal to zero yields:

$$\boldsymbol{\mu}_1 = \frac{1}{N_1} \sum_{n=1}^N t_n \mathbf{x}_n$$

which is simply the mean of all input vectors $\mathbf{x}_n$ assigned to $\mathcal{C}_1$. Correspondingly, for $\boldsymbol{\mu_2}$

$$\boldsymbol{\mu}_2 = \frac{1}{N_2} \sum_{n=1}^N (1-t_n) \mathbf{x}_n,$$

the mean of all the input vectors assigned to $\mathcal{C}_2$. This also makes sense.

Finally, consider the maximum likelihood solution for the shared covariance matrix $\boldsymbol{\Sigma}$. The terms that depend on $\boldsymbol{\Sigma}$ in the log likelihood are:

$$-\frac{1}{2} \sum_{n=1}^{N} t_n \ln |\boldsymbol{\Sigma} | - \frac{1}{2} \sum_{n=1}^N t_n(\mathbf{x}_n - \boldsymbol{\mu_1})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}_n - \boldsymbol{\mu}_1) $$
$$-\frac{1}{2} \sum_{n=1}^N (1-t_n) \ln |\boldsymbol{\Sigma} | -\frac{1}{2} \sum_{n=1}^N (1-t_n) (\mathbf{x}_n - \boldsymbol{\mu_2})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}_n-\boldsymbol{\mu_2})$$

$$= -\frac{N}{2} \ln | \boldsymbol{\Sigma} | -\frac{N}{2} Tr \{ \boldsymbol{\Sigma}^{-1} \mathbf{S} \}$$

where

$$\mathbf{S} = \frac{N_1}{N} \mathbf{S}_1 + \frac{N_2}{N} \mathbf{S}_2$$

$$\mathbf{S}_1 = \frac{1}{N_1} \sum_{n \in \mathcal{C}_1} (\mathbf{x}_n - \boldsymbol{\mu_1}) (\mathbf{x}_n - \boldsymbol{\mu}_1)^T $$

$$\mathbf{S}_2 = \frac{1}{N_2} \sum_{n \in \mathcal{C}_2} (\mathbf{x}_n - \boldsymbol{\mu_2}) (\mathbf{x}_n - \boldsymbol{\mu}_2)^T $$

Taking the derivative and rearranging yields $\boldsymbol{\Sigma} = \mathbf{S}$, i.e., the weighted average of the covariance matrices associated with each of the two classes separately.

This result is easily extended to the $K$-class problem. However, this formulation will have the same problem with outliers that the discriminant function did, because maximum likelihood estimation of a Gaussian is not robust to outliers, as we have seen above.

## Bayes Classifier

Let's look at some specific generative classifiers.

Bayes theorem for classification looks like this:

$$p(y_k|\mathbf{x_i}) = \frac{p(\mathbf{x}_i | y_k) p(y_k)}{\sum_i p(\mathbf{x}_i | y_k) p(y_k)}.$$

Here, $p(y_k)$ si the probability of any point having class $k$ regardless of which point it is, i.e., the prior probability of class $k$.  For example, this might be the fraction of the training set that is in class $k$.  

The task of "learning the classifier" is then the task of determining the $p_k$ densities.  

Consider the regression function $\hat y = f(y|\mathbf{x})$ as the best guess of $y$ given a specific value of $\mathbf{x}$.  Consider that $y$ can take on only the values 0 and 1.  Then $f(y|\mathbf{x})$ is the discriminant function:

$$g(\mathbf{x}) = f(y|\mathbf{x})=\int yp(y|\mathbf{x}) dy$$
$$= 1\centerdot p(y=1|\mathbf{x})+0\centerdot p(y=0|\mathbf{x})=p(y=1 |\mathbf{x})$$

Now apply Bayes rule:

$$g(\mathbf{x}) = \frac{p(\mathbf{x}|y=1)p(y=1)}{p(\mathbf{x}|y=1)p(y=1)+p(\mathbf{x}|y=0)p(y=0)}$$
$$ =\frac{\pi_1 p_1(\mathbf{x})}{\pi_1 p_1(\mathbf{x}) + \pi_0 p_0(\mathbf{x})}.$$

The Bayes classifier is formulated so that 

$$\hat y = \begin{cases}
1 & \text{if } g(\mathbf{x})>1/2,\\
0 & \text{otherwise,}
\end{cases}$$
$$= \begin{cases}
1 & \text{if }p(y=1|\mathbf{x}) > p(y=0|\mathbf{x}),\\
0 & \text{otherwise,}
\end{cases}$$
$$= \begin{cases}
1 & \text{if }\pi_1 p(\mathbf{x}) > \pi_0 p_0(\mathbf{x}),\\
0 & \text{otherwise}
\end{cases}$$

This can be generated to many classes $K$, where there will be a $g_k(\mathbf{x})$ for each class.

In this case, the decision boundary is located where the two classes are equally likely

$$\pi_1 p_1(\mathbf{x}) = \pi_2 p_2(\mathbf{x}).$$

### Naive Bayes

The Bayes classifier is conceptually simple, but can be difficult to compute, as the data may have many dimensions and may have complicated probability distributions.

Make the simplifying assumption that attributes that are measured are independent so that

$$p(x^i,x^j|y_k)=p(x^i|y_)p(x^j|y_k),$$

where the superscript indexes features of the vector $\mathbf{x}$.   

In many dimensions, this becomes:

$$p(x^0,x^1,x^2,\dots,x^N|y_k) = \prod_i p(x^i|y_k).$$

Apply Bayes rule to obtain:

$$p(y_k|x^0,x^1,\dots,x^N)=\frac{p(x^0,x^1,\dots,x^N|y_k)p(y_k)}{\sum p(x^0,x^1,\dots,x^N|y_j)p(y_j).}$$

Applying the independence above, this becomes:

$$p(y_k|x^0,x^1,\dots,x^N)=\frac{\prod_i p(x^i|y_k)p(y_k)}{\sum_j \prod_k p(x^i|y_j)p(y_j)}.$$

We obtain the most likely value of $y$ by maximizing over $y_k$, 

$$\hat y=\text{arg }\text{max}_{y_k}=\frac{\prod_i p(x^i|y_k)p(y_k)}{\sum_j \prod_k p(x^i|y_j)p(y_j)},$$

where [$\text{arg }\text{max}$](https://en.wikipedia.org/wiki/Arg_max) is a mathematical way of specifying the position $y_k$ where the argument of it obtains a maximum.

or if we use $\pi_k=p(y=y_k)$, we write:

$$\hat y = \text{arg }\text{max}_{y_k}=\frac{\prod_i p(x^i)\pi_k}{\sum_j \prod_k p(x^i)\pi_j},$$

So, once $p_k(x^i)$ and $\pi_k$ are known, then $\hat y$ can be computed.  

Finding $p_k(x^i)$ and $\pi_k$ is the challenge.  According to Ivezic, it can be accomplished in various ways on a training set, using methods in Chapter 4 (Classical Statistical Inference), Chapter 5 (Bayesian Statistical Inference), Chapter 6 (Searching for Structure in Point Data, which include density estimation techniques).

But in the case where the information is categorical, i.e., $x^i$ in the training set tells simply about what class the object is in, then:
 - for each label $y_k$ in the training set, the maximum likelihood estimate of the probability for a feature $x^i$ is equal to the number of objects with a particular value of $x^i$ divided by the total number of objects with $y=y_k$.  
 - The prior probabilities are given by the fraction of the training data with $y=y_k$.  

### Gaussian Naive Bayes and Gaussian Bayes Classifiers

Rarely are there discrete values of $\mathbf{x}$ even when there are categorical labels for $y$.  More likely, $\mathbf{x}$ will be continuous.  But we can still use the method above, if we assume that $p_k(x^i)$ is modeled as a 1-dimensional normal distribution with means $\mu_k^i$ and widths $\sigma_k^i$ determined using, for example, the usual frequentist techniques.  Then the estimator becomes (i.e., by taking the log, since the maximum will occur at the same point as for the argument and the log of the argument):

$$\hat y =\text{arg }\text{max}_{y_k} 
\left[ \ln \pi_k - \frac{1}{2} \sum_{i=1}^N \left (2 \pi (\sigma_k^i)^2+\frac{(x^i-\mu^i_k)^2}{(\sigma_k^i)^2} \right ) \right ].$$

This is possibly an unrealistically simple case, but let's see how it works below.

In this example, $X$ has two attributes.  Half of $X$ are mapped to target $y=0$ and the other half are mapped to $y=1$.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
from sklearn.naive_bayes import GaussianNB


In [ ]:
# create some fake data that has two normal distributions.   
# Assign the one normal distribution to a target value zero, the other to a target value 1.
# Note that this does not use the 1-in-k encoding mentioned above.

np.random.seed(0)
mu1 = [1, 1]
cov1 = 0.3 * np.eye(2)

mu2 = [5, 3]
cov2 = np.eye(2) * np.array([0.4, 0.1])

X = np.concatenate([np.random.multivariate_normal(mu1, cov1, 100),
                    np.random.multivariate_normal(mu2, cov2, 100)])
y = np.zeros(200)
y[100:] = 1


In [ ]:
print X.shape
print y.shape

plt.plot(X[:100,0],X[:100,1],'o')
plt.plot(X[100:,0],X[100:,1],'o')

In [ ]:
# Fit the Naive Bayes classifier
clf = GaussianNB()
clf.fit(X, y)

# predict the classification probabilities on a grid.   
xlim = (-1, 8)
ylim = (-1, 5)
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 71),
                     np.linspace(ylim[0], ylim[1], 81))

#predict_proba requires the data to be entered as vectors of numbers, i.e., two 
#locations on the grid.  So the numpy ravel returns a contiguous flattened array,
#which is reshaped afterwards.
Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z = Z[:, 1].reshape(xx.shape)


In [ ]:
# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[:100,0],X[:100,1],'o')
plt.plot(X[100:,0],X[100:,1],'o')

# Z ranges between zero and 1 depending on the region. So the 
# decision boundary is located between zero and 1.

plt.contour(xx,yy,Z,[0.5],colors='k')



In [ ]:
print Z.shape
plt.plot(Z[:,35])

$Z$ in the example above is a map of values inferred from $X,y$ that classifies the whole space with a probability of either being in the first class or the second class.  The boundary between them can be identified as having probability of 0.5.  So the contour in the plot above is set to 0.5.

The figure below (Figure 9.3) shows the application of the naive Bayes classifier to the RR Lyrae data set.  It separates variable RR Lyrae stars from non-variable main sequence stars.  

![Figure 9.3 from Ivezic](http://www.astroml.org/_images/fig_rrlyrae_naivebayes_1.png)

The left panel shows the classification boundary given by the black line, with the probability given by the shaded region.  The light grey points show the variable points, while the dark points show variable sources.  In the right panel, the "completeness" (fraction of total detections identified by the classifier) and the "contamination" (fraction of detected objects that are misclassified) are shown as a function of the number of attributes used in the classification.

scikit-learn has also implementations of Multinomial Naive Bayes and Bernoulli Naive Bayes.  The interested student can find information about these programs [here](http://scikit-learn.org/stable/modules/naive_bayes.html).

### Linear and Quadratic Discriminant Analysis

Linear discriminant analysis (LDA) also makes some simplifying assumptions about the class distributions $p_k(\mathbf{x})$ in our general equation: 

$$p(y_k|\mathbf{x_i}) = \frac{p(\mathbf{x}_i | y_k) p(y_k)}{\sum_i p(\mathbf{x}_i | y_k) p(y_k)}.$$

It assumes that the distributions have identical covariances for all $K$ classes, so that all classes are sets of shifted Gaussians.  Then the optimal classifier can be obtained from the log of the class posteriors to be

$$g_k(\mathbf{x})=\mathbf{x}^T\Sigma^{-1}\mu_k-\frac{1}{2}\mu_k^T\Sigma^{-1}+\log \pi_k,$$

where $\mu_k$ is the mean of class $k$ and $\Sigma$ is the covariance of the Gaussians.  The Bayes classifier ends up being linear with respect to $\mathbf{x}$ if the distributions all have identical covariances.  The discriminant boundary ends up being a line that minimizes the overlap between Gaussians.

$$g_k(\mathbf{x})-g_l(\mathbf{x})=\mathbf{x}^T \Sigma^{-1}(\mu_k-\mu_l)-\frac{1}{2}(\mu_k-\mu_l)^T \Sigma^{-1} (\mu_k-\mu_l) + \log\left( \frac{\pi_k}{\pi_l}\right )=0.$$



If we relax the requirement that the covariance of the Gaussians are constant,  i.e., each cluster has its own covariance, the discriminant function becomes quadratic in $x$:

$$g(\mathbf{x})=-\frac{1}{2}\log \lvert \Sigma_k \rvert - \frac{1}{2}(\mathbf{x}-\mu_k)^TC^{-1} (\mathbf{x}-\mu_k)+\log \pi_k.$$

This is called **quadratic discriminant analysis** (QDA), and the boundary between the classes will be a quadratic function of $\mathbf{x}$.  

Let's try these classifiers on the data set above.  (Note that the examples from the book website don't work as written; the LDA & QDA have been depreciated (or something).)  So instead we use [LinearDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html) and [QuadraticDiscriminantAnalysis](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html).

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)

lda.fit(X,y)

In [ ]:
Z_lda = lda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
Z_lda = Z_lda[:, 1].reshape(xx.shape)

# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[:100,0],X[:100,1],'o')
plt.plot(X[100:,0],X[100:,1],'o')

plt.contour(xx,yy,Z_lda,[0.5],colors='k')




In [ ]:
## Try quadratic discriminant analysis

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

#qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda = QuadraticDiscriminantAnalysis()



qda.fit(X,y)

In [ ]:
Z_qda = qda.predict_proba(np.c_[xx.ravel(), yy.ravel()])
print Z_qda.shape
Z_qda = Z_qda[:, 1].reshape(xx.shape)

# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[:100,0],X[:100,1],'o')
plt.plot(X[100:,0],X[100:,1],'o')

plt.contour(xx,yy,Z_qda,[0.5],colors='k')





Wondering how these do when there are multiple target values, say 3 of them, but still two X-directions for ease in plotting.

In [ ]:
# create some fake data.

np.random.seed(0)
mu1 = [-1, 1]
cov1 = 0.1 * np.eye(2)

mu2 = [-1, -1]
cov2 = 0.1 * np.eye(2)

mu3 = [1.5,0]

cov3 = np.eye(2) * np.array([0.1,0.4])
#cov3 = 0.1 * np.eye(2)


X = np.concatenate([np.random.multivariate_normal(mu1, cov1, 100),
                    np.random.multivariate_normal(mu2, cov2, 100),
                    np.random.multivariate_normal(mu3, cov3, 100)])
y = np.zeros(300)
y[100:200] = 1.0
y[200:] = 2.0


In [ ]:
plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1.0,0],X[y==1.0,1],'o')
plt.plot(X[y==2.0,0],X[y==2.0,1],'o')

In [ ]:
lda_three = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)

lda_three.fit(X,y)

In [ ]:
xlim = (-2.5, 3)
ylim = (-2.5, 2.5)

xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 71),
                     np.linspace(ylim[0], ylim[1], 81))

Z_lda_three = lda_three.predict_proba(np.c_[xx.ravel(), yy.ravel()])
print Z_lda_three.shape
Z_lda_three_part1 = Z_lda_three[:, 1].reshape(xx.shape)
Z_lda_three_part2 = Z_lda_three[:, 2].reshape(xx.shape)


# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')
plt.plot(X[y==2,0],X[y==2,1],'o')

plt.contour(xx,yy,Z_lda_three_part1,[0.5],colors='k')
plt.contour(xx,yy,Z_lda_three_part2,[0.5],colors='k')


print Z_lda_three.max()

In [ ]:
qda_three = QuadraticDiscriminantAnalysis()

qda_three.fit(X,y)

In [ ]:
Z_qda_three = qda_three.predict_proba(np.c_[xx.ravel(), yy.ravel()])
print Z_qda_three.shape
Z_qda_three_part1 = Z_qda_three[:, 1].reshape(xx.shape)
Z_qda_three_part2 = Z_qda_three[:, 2].reshape(xx.shape)


# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')
plt.plot(X[y==2,0],X[y==2,1],'o')

plt.contour(xx,yy,Z_qda_three_part1,[0.5],colors='k')
plt.contour(xx,yy,Z_qda_three_part2,[0.5],colors='k')




##  Probabilistic Discriminative Models

In the last section, we showed how the posterior probability of a class can be written as a logistic sigmoid acting on a linear function of $\mathbf{x}$ for the two-class problem. Likewise, we have seen how the softmax transformation works in the same way for the $K$-class situation. Once we have the $p(\mathbf{x} | \mathcal{C}_k)$, we could use maximum likelihood to determine the parameters of the densities, then with Bayes' theorem, we could find the posterior class probabilities.

In this section, we will examine an alternative approach: we will use the functional form of the generalized linear model explicitly, and determine its parameters directly using maximum likelihood.  More specifically, we will dispense with computing the probabilities, and classify the data directly.  



### Logistic Regression

Logistic regression is a very commonly used, mainstream classification technique.  It is emphasized that although the name includes the word regression, this is a model for classification rather than regression.

Consider again the case where there are two classes, $\mathcal{C}_1$ and $\mathcal{C}_2$. The posterior probability of class $\mathcal{C}_1$ can be written as a logistic sigmoid acting on the linear function of the feature vector $\phi$ as 

$$p(\mathcal{C}_1 | \phi) = y(\phi) = \sigma(\mathbf{w}^T \phi)$$

with $p(\mathcal{C}_2 | \phi) = 1-p(\mathcal{C}_1 | \phi)$. So this is the same as we had above, except now instead of working on the data $\mathbf{x}$, we are working on the sigmoid function $\sigma$.   This model is known as logistic regression.

Now, as usual, use maximum likelihood to determine the parameters of the logistic regression model. 

We will make use of the relationship

$$\frac{d\sigma}{da} = \sigma(1-\sigma),$$

where recall that 

$$a=\ln \frac{p(\mathbf{x} | \mathcal{C}_1)p(\mathcal{C}_1)}{p(\mathbf{x}|\mathcal{C}_2 p(\mathcal{C}_2)}$$

and 

$$\sigma(a)=\frac{1}{1+\exp(-a)}.$$

Then, for a data set $\{\phi_n,t_n\}$, where $t_n \in \{0,1\}$, and $\phi_n = \phi(\mathbf{x}_n)$, with $n=1,\ldots,N$, the likelihood function is written as

$$p(\mathbf{t} | \mathbf{w}) = \prod_{n=1}^N y_n^{t_n} \{1-y_n\}^{1-t_n}$$

where $\mathbf{t} = (t_1,\ldots,t_N)^T$, and $y_n=p(\mathcal{C}_1 |
\phi_n)$.

In this case, $y$ is binary (we have only two classes so far), so this is a Bernoulli function.

Define an error function by taking negative the logarithm of the likelihood, which gives the so-called cross-entropy error function in the form

$$E(\mathbf{w}) = - \ln p(\mathbf{t} | \mathbf{w})$$
$$ = -\sum_{n=1}^N \{t_n \ln y_n + (1-t_n) \ln (1-y_n)\}$$

where $y_n = \sigma(a_n)$ and $a_n=\mathbf{w}^T \boldsymbol{\phi}_n$. Taking the gradient of the error function with respect to $\mathbf{w}$, we obtain

$$\nabla E(\mathbf{w}) = \sum_{n=1}^N (y_n - t_n) \boldsymbol{\phi}_n.$$

A few things to note.

We have kept our promise - we are working directly to a solution, without assuming a Gaussian or other shape for $p(\mathbf{x}_n,\mathcal{C}_1)$, as before.

The contribution of the data point to the gradient is $y_n - t_n$, i.e., something like the difference between the target $t_n$ (which here is a class identification) and the predicted value, remembering that $p(\mathcal{C}_1 | \phi) = y(\phi) = \sigma(\mathbf{w}^T \phi)$.  

Also note that the form of the equation is the same as one we derived in the regression section.
It is also worth noting that several of these formulae will arise again when we talk about neural networks, which can be thought of as repeated applications of the sigmoid function (or other activation function).

### Multiclass Logistic Regression

Not surprisingly, we can generalize to the case of $K > 2$ classes. The posterior probabilities will be given by softmax transformations of linear functions of the input variables, so that

$$p(\mathcal{C}_k | \phi) = y_k(\boldsymbol{\phi}) = \frac{\exp (a_k)}{\sum_j \exp (a_j)}$$

where the activations $a_k$ are given by

$$a_k = \mathbf{w}^T_k \boldsymbol{\phi}.$$

Previously, we used maximum likelihood to determine separately the class-conditioned densities and the class priors and then found the corresponding posterior probabilities using Bayes' theorem, with the result being the parameter $\{w_k\}$. For logisitic retression, we use maximum likelihood to determine $\{w_k \}$ directly.

To do this, we  require the the derivatives of $y_k$ with respect the activations $a_j$:

$$\frac{\partial y_k}{\partial a_j} = y_k (I_{kj}-y_i)$$

where $I_{kj}$ are the elements of the identity matrix.

Next, write down the likelihood function using the 1-of-$K$ coding scheme in which the target vector $\mathbf{t}_n$ for a feature vector $\phi_n$ belonging to class $\mathcal{C}_k$ is a binary vector with all elements zero except for element $k$, which equals 1. Then

$$p(\mathbf{T} | \mathbf{w}_1,\ldots,\mathbf{w}_k) = \prod_{n=1}^N \prod_{k=1}^K p(\mathcal{C}_k | \boldsymbol{\phi_n})^{t_{nk}} = \prod_{n=1}^N \prod_{k=1}^K y_{nk}^{t_{nk}}$$

where $y_{nk} = y_k(\phi_n)$, and $\mathbf{T}$ is an $N \times K$ matrix of target variables with elements $t_{nk}$. Taking the negative logarithm, as usual, yields

$$E(\mathbf{w}_1,\ldots,\mathbf{w}_k) = -\ln p(\mathbf{T} | \mathbf{w}_1,\ldots,\mathbf{w}_K) = -\sum_{n=1}^N \sum_{k=1}^K t_{nk} \ln y_{nk}.$$

This is known as the cross-entropy error function for the multi class classification problem. As usual, we next take the gradient of the erorr function with respect to one of the parameter vectors $\mathbf{w}_j$. Making use of the softmax derivative result above yields

$$\nabla_{\mathbf{w}_j} E(\mathbf{w}_1,\ldots,\mathbf{w}_k) = \sum_{n=1}^N (y_{nj} - t_{nj}) \boldsymbol{\phi}_n$$

where use has been made of the fact that $\sum_k t_{nk} = 1$.

This is, not surprisingly, quite similar to what we saw before, and it can be solved in a similar way.

scikit-learn has an implementation of logistic regression described [here](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression).

Let's try to use this on the three-class data generated above.

In [ ]:
from sklearn.linear_model import LogisticRegression



In [ ]:
clf_logit = LogisticRegression(solver='sag', max_iter=100, random_state=42,
                             multi_class='multinomial').fit(X, y)



In [ ]:
Z_logit=Z = clf_logit.predict(np.c_[xx.ravel(), yy.ravel()])
print Z_logit.shape
Z_logit = Z_logit.reshape(xx.shape)
print Z_logit.shape

print Z_logit.max()

In [ ]:
# Plot the results

plt.xlabel('X[:,0]',fontsize=20)
plt.ylabel('X[:,1]',fontsize=20)

plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')
plt.plot(X[y==2,0],X[y==2,1],'o')

plt.contour(xx,yy,Z_logit,[0.5],colors='k')
plt.contour(xx,yy,Z_logit,[1.5],colors='k')


print Z_lda_three.max()

## The Perceptron

Marsland presents a nice inituitive discussion of the perceptron in his Chapter 3, including some historical development.  

They start with a single neuron which consists of:
 - a set of inputs $\mathbf{x}$, which are weighted by the weights $\mathbf{w}$.
 - an adder, that sums the input signals, illustrated by the $\Sigma$ below.
 - an activation function that concludes whether the neuron will "fire" based on the result of the adder.
 
Optionally, there is is a "bias", $w_0$ which is a constant offset that allows the activation function to fire at a value other than zero (if desired).  

![perceptron wikipedia](https://upload.wikimedia.org/wikipedia/commons/8/8c/Perceptron_moj.png)

So the idea is that various signals will come in, they will be summed using the weights, and then if the value exceeds a threshold, the neuron will fire.

The Perceptron is simply a network of neurons, consisting of an input array, but allowing multiple connections to exist so that there can be multiple outputs.

Marsland also includes a nice discussion of how a neuron learns with a training set.  Recall that a training set will consist of the inputs $\mathbf{x}$ and the target values $\mathbf{t}$
 - Choose some random values for the weights.
 - Input a training value and see what value is produced for those weights.
 - Compare the output values $\mathbf{y}$ with the target values.
 
Some of the outputs may match the target values, and some may be incorrect.  The weights will need to be adjusted until they are all correct (this is the process of learning).  

To do this, first compute $y_k-t_k$, and then update the weight that connects input $i$ with output $j$ as 

$$w_{ij} - \eta(y_j - t_j)\centerdot x_i,$$

where $\eta$ is the _learning rate_, which decides how fast the network learns.    This is an input parameter that specifies how to much to change the weights by. For me, it has an analogy with "$\Delta$" in spectral fitting, i.e., the amount to change parameter in the quest for a minimum $\chi^2$.  If you change the parameter by too much, then the next step may overshoot the minimum, if you change it by too little, it will never converge.  Marsland comments that a moderate learning rage, with $0.1 < \eta < 0.4$, may serve.

Marsland describes a simple example: that of a simple "OR" logic gate, which is described by:

In1 | In2 | t
----|-----|----
0 | 0 | 0
0 | 1 | 1
1 | 0 | 1
1 | 1 | 1

This network has two inputs, a bias that is set equal to $-1$ and a single output.
 - Start the weights at random small numbers, e.g., $w_0 = -0.05$, $w_1=-0.02$, $w_2=0.02$.
 - Feed in the first input set: $(0,0)$.  The value that will reach the neuron is 0.05.  This is above zero, so the neuron will fire and the output will be $1$, which is not correct.
 - So apply the update rule for each of the three $w$, using $\eta=0.25$, and yielding $\mathbf{w}=(0.2, -0.02, 0.02)$
 - Feed in the next input set, $(0,1)$.  The neuron should fire, but it doesn't, so you need to update again.
 - Iterate until it coverges, passing through the whole set of inputs again, until you get values that work.
 
Note that you don't care in fact what the $w$s are, just that you produce the correct output for of the possible inputs.  The actual values of $\mathbf{w}$ may depend on the starting values.


How do we relate the perceptron to what we have already discussed?

It turns out that the perceptron will work if there is "linear separability" between the two classes, then it will work.  In fact, what it does is what we have done above: found straight line (2D), plane (3D), or hyperplane between the classes, i.e., the decision boundary, exactly as above.

It will fail if there is no such line.  The example that Marsland gives is that of the "XOR" logic gate, i.e,. the "exclusive or".  The exclusive or returns a 1 when the two inputs are different, but returns a zero if they are the same.  What do do in this case?

## The Multi-layer Perceptron

The problem of the XOR logic gate can be addressed using a multi-layer perceptron.  A multi-layer perceptron is one has one or more layers of nodes between the input and the output nodes.  The interior layers are called hidden layers, and in principle there can be many layers, although we will discuss only one hidden layer.  

Marsland shows that the XOR gate works with the network in his figure 4.2, sketched below, and you can work through the options to see that it does for  yourself.

![marsland fig 4.2 sketch](http://www.nhn.ou.edu/~leighly/marsland_4.2_sketch.tiff)

But how to train the multi-layer perceptron?  One can use the error, as above, to correct the weights, but it is not possible using that method to determine which weights (the ones from the input to the hidden layer, or the ones from the hidden layer to the output) are required to be adjusted.  This problem was apparently solved in 1986, and according to Marsland, the multi-layer perceptron is one of the most commonly used machine learning methods, and one of the most common neural networks.  

As above, we will discuss how the training is done, and then we will go through the math a la Bishop.

### Forward training

Forward training goes just the same as with the single neuron above.  The weights are randomly chosen small values.  The input of the training set and the first level of weights is used to compute the activations of the hidden layer, and those activations are used with the next set of weights to get the predicted values $y_k$.  These can then be compared with the training set target values to compute an error.

### Back-Propagation of Error

The error will be propagated back through the network to compute the updates for the weights.  The method used is called _back-propagation of error_.  It uses a form of gradient descent to compel the errors toward a minimum.  

Gradient descent is very common and intuitive data fitting method.  It uses the derivative to determine a gradient, and the problem is iterated toward a solution.

The error used will be the sum of squares error function:

$$E(\mathbf{t},\mathbf{y})=\frac{1}{2} \sum_{k=1}^N (y_k-t_k)^2.$$

This will be differentiated with respect to the weights.

We must also compute the derivative of the activation function.  This presents a problem, therefore, with the Heaviside function used above, as it is not differentiable.  So the sigmoid or $\tanh$ function is used instead.

In the example above, where we had only one layer, it was easy to compare the output with the weights.  Here it is more difficult because we don't know either the input or output to the hidden layer.  So instead, the chain rule is used to write an equation that brings us all the way from the output back to the weights at all levels.  

So here is a summary of the procedure, adopted from Marsland:

1. an input vector is put into the input nodes
2. the inputs are fed forward through the network.
    - the inputs and first-layer weights are used to decide whether the hidden nodes fire or not.  
    - the outputs of these neurons and the second-layer weights are used to decide if the output neurons fire or not.
3. the error is computed as the sum-of-squares difference between the outputs and the targets.
4. This eror is fed backward through the network to 
    - first update the second-layer weights
    - next update the first-layer weights.

Marsland makes a few more useful comments (he also provides examples and further discussion).

#### Initializing the weights

Above it instructs to initalize the weights to small random numbers.  Why is that?  It is because of the shape of the sigmoid.  The sigmoid changes rapidly near the value of zero, and is saturated far from that value.  So small weights, regardless of the input, will yield a rapidly changing output. This allows the network to be trained faster.  Apparently, the common trick is to allow the weights to lie in the range $-1/\sqrt{n} < w < 1/\sqrt{n}$, where $n$ is the number of nodes in the input layer.  In addition, all the weights are randomize to be about the same size so that all weights reach their final values at the same time.

#### Scaling the Data

If the weights are chosen as above, you will not want either your input or your output to be much larger / smaller than $1$ or $-1$.  So the data needs to be suitably scaled, e.g., by subtracting the mean and dividing by the standard deviation (which will produce data with a mean of zero and a standard deviation of 1).

#### Sequential and Batch Training

While the algorithm is described as a sequential process, the MLP (Multi-Layer Perceptron) is trained faster if the computations are done in batch by layers.

#### Local Minima

Gradient descent can end up getting stuck in local minima.  One simple way to combat this is to run the training several times with different starting points. And there are methods to address this problem; see Marsland.

#### Amount of training data

One MLP with one hidden layer will require $(L+1)*M + (M+1)*N$ weights, where $L$, $M$, $N$ are the number of nodes in the input, hidden and output layers.  (The extra "+1"s come from the biases).    The rule of thumb is to use a training set has 10 times the number of weights.

#### Number of Hidden Layers

There is a theorem called the "Universal Approximation Theorem" which states that one hidden layer with lots of nodes is sufficient. 

#### When to Stop Learning

You'll have to iterate through the forward and backward procedures several times to learn the weights.  But MLP is apparently subject to overfitting, where the weights will start to learn the noise, rather than just the data.  To address this, we can use the usual techniques, such as regularization.  Marsland discusses using cross validation, i.e., run the process on a training set and a cross validation set, and examine the square error for both sets as a function of iteration.  Both will decrease together as the training proceeds, but eventually the cross validation error will start to increase as the weights start to model the noise (as the exact noise pattern will be unique for each set).  So stop when that starts to happen.

## Neural Networks

Chapter 5 in Bishop covers neural networks. 

As discussed previously in our motivation for Gaussian processes, usually the basis functions are fixed.  

The idea behind this chapter is that the number of basis functions is fixed, but they are allowed to be adaptive, i.e., the basis functions have parametric forms but the parameter values are adapted during training.  This results in considerable flexibility.  For example decision boundaries in logistic regression are either straight lines or parabolas.  Neural network decision boundaries can have complicated shapes.

The specific model that will be discussed is the feed-forward neural network, i.e., the multilayer perceptron. In fact, the model comprises multiple layers of logistic regression models.

#### Feed-forward Network Functions

The linear models for regression and classification that we have discussed so far are based on linear combinations of fixed nonlinear basis functions $\phi_j(\mathbf{x})$ and take the form

$$y(\mathbf{x},\mathbf{w}) = f \left ( \sum_{j=1}^M w_j \phi_j (\mathbf{x}) \right )$$

where $f$ is a nonlinear activation function in the case of classification and is the identity in the case of regression. The idea, as stated above, is to extend the model by making the basis functions $\phi_j (\mathbf{x})$ depend on parameters, and allow these parameters to be adjusted, along with the coefficients $\{ w_j \}$, during training. Neural networks use basis functions that follow the same form as the equation above, i.e., each basis function is a nonlinear function of a linear combination of the inputs, and the coefficients in the linear combination are adaptive parameters.

The basic neural network can be described as a series of functional transformations. 

First, construct $M$ linear combinations of the input variables $x_1,\ldots,x_D$ in the form

$$a_j = \sum_{i=1}^D w_{ji}^{(1)} x_i + w_{j0}^{(1)}$$

where $j=1,\ldots,M$, and the superscript $(1)$ indicates the parameters are in the first layer of the network. We make some definitions:

- The parameters $w_{ji}^{(1)}$ are referred to as _weights_.
- The parameters $w_{j0}^{(1)}$ are known as _biases_.
- The quantities $a_j$ are known as _activations_.

Next, each of the activations are transformed using a differentiable, nonlinear activation function $h$ to give

$$z_j = h(a_j).$$

In the context of neural networks, the $z_j$ are called _hidden units_.

The nonlinear functions $h$ are generally chosen to be sigmoidal functions such as the _logistic sigmoid_ or the _tanh_ function.

Next, these values are linearly combined to give output unit activations

$$a_k = \sum_{j=1}^M w_{kj}^{(2)} z_j + w_{k0}^{(2)}$$

where $k=1,\ldots,K$, and $K$ is the total number of outputs. This transformation corresponds to the second layer of the network.

Finally, the output unit activations are transformed using an appropriate activation function to give a set of network outputs $y_k$. The choice of activation function is determined by the data.  For standard regression problems, the activation function is the identity so that $y_k=a_k$. For binary classification problems, each output unit activation is transformed using the logistic sigmoid function, so that:

$$y_k = \sigma(a_k)$$

where

$$\sigma(a) = \frac{1}{1+\exp(-a)}.$$

For multiclass problems, the softmax function is used.

So, all of these stages can be combined to give (for sigmoidal output function)

$$y_k(\mathbf{x},\mathbf{w}) = \sigma \left ( \sum_{j=1}^M w_{kj}^{(2)}
h \left (\sum_{i=1}^D w_{ji}^{(1)} x_i + w_{j0}^{(1)} \right )
+w_{k0}^{(2)} \right ) $$

where the set of all weight and bias parameters are grouped together into a single vector $\mathbf{w}$. 

So the neural network model is a nonlinear function from a set of input variables $\{ x_i \}$ to a set of output variables $\{ y_k \}$ controlled by a vector $\mathbf{w}$ of adjustable parameters.

A diagram of the neural net model is seen below.

![wikipedia](https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/499px-Colored_neural_network.svg.png)

A briefer form of the above equation can be written, by absorbing the bias parameters into the weight parameters by defining the additional parameter $x_0 =1$, and performing the same trick with the hidden variable $z_0$. Then

$$y_k (\mathbf{x}, \mathbf{w}) = \sigma \left ( \sum_{j=0}^M w_{kj}^{(2)} h \left ( \sum_{i=0}^D w_{ji}^{(1)} x_i \right ) \right ).$$

Bishop makes a number of points about this network.

- If the activation functions of the hidden units are linear, then the result is equivalent to a linear problem without the hidden units.  But as a result, flexibility that you get using sigmoidal activation functions is lost.  Moreover, it turns out that networks of linear units give rise to principal components analysis.
- The network shown is not the only one possible. There can be more layers, for example. There can also be generalizations where some connections skip a layer, or don't have connections between every node. 
- The one thing that should be required is that the network has a _feed-forward architecture_, so that there are no closed cycles, and the outputs are deterministic functions of the inputs.


The two-layer network is very flexible. For example, it can be used to approximate any continous function provided there are enough hidden units.   This result holds for a range of hidden unit activation functions, except for polynomials. This is illustrated in Bishop Figure 5.3, which shows a parabola, a cubic, $f(x)=|x|$, and a Heaviside function reproduced by sigmoid basis functions using the multilayer perceptron. 


### Network Training

Now we understand what a neural network does, we have to figure out how to do it, i.e., how to use the training data to determine the $w$s. Basically, it is the same story as before. 

Bishop first considers the simple least squares formulation. Given a training set comprised of $\{ \mathbf{x}_n \}$, where $n=1,\ldots,N$, and a corresponding set of target vactors $\{ \mathbf{t}_n \}$, we need to minimize the error function

$$E(\mathbf{w}) = \frac{1}{2} \sum_{n=1}^N \lVert \mathbf{y}(\mathbf{x}_n,\mathbf{w}) - \mathbf{t}_n \rVert ^2.$$

But, as usual, we are not satisfied with least squares, we expect that it will perform poorly when there are outliers, for example, so we would like to instead consider a probabilistic approach. A lot of this is quite repetitive and I will just outline it.

- Consider first regression. Write $p(\mathbf{t} | \mathbf{X},\mathbf{w}, \beta)$ as the product of the normal distributions of individual outcomes $t$. Take the negative logarithm to get the error function. Take the derivatives & set to zero to get the maximum likelihood $\mathbf{w}_{ML}$ and $\beta_{ML}$.
- Next consider classification. Now, the logistic sigmoid comes into play (or softmax function if there are more than two classes). Work through the same as before to get the cross-entropy error function.




#### Parameter Optimization

Generally, this will not be expressible in closed form, so numerical methods will be necessary to obtain the weight vector $\mathbf{w}$ that minimizes the error function $E(\mathbf{w})$.  The one that seems to be used frequently is something related to [_gradient decent_](https://en.wikipedia.org/wiki/Gradient_descent).

One can imagine a concave surface in weight space.

If one makes a small step in weight space from $\mathbf{w}$ to $\mathbf{w}+\delta \mathbf{w}$, then the change in the error function is $\delta E \simeq \delta \mathbf{w}^T \nabla E(\mathbf{w})$, where $\nabla E(\mathbf{w})$ points in the direction of greatest increase of the error function. Naturally, the minimum will be found when the gradient of the error function vanishes (the stationary points), and

$$\nabla E(\mathbf{w}) = 0,$$

otherwise, a small step in the directin of $-\nabla E(\mathbf{w})$ would reduce the error.

The goal is to find $\mathbf{w}$ that yields the smallest value of $E(\mathbf{w})$. However, there will generally be multiple stationary minima, due to the nonlinearity of the functions, but also because of certain symmetries in the problem, e.g., the fact that hidden units can be switched with one another, or the fact that the $\tanh$ function is symmetric. So there will be local minima. Bishop states that it is not necessary to find the global minimum  but several local minima should be compared to find a sufficiently good solution (i.e., the solution should be run several times with different starting points.)

An analytic solution to $\nabla E(\mathbf{w})$ is intractable, so an interative numerical procedure is used, along the lines of

$$\mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} + \Delta
\mathbf{w}^{(\tau)}$$

where $\tau$ labels the iteration step. Different algorithms may be used, but many will use the gradient information. This approach is quite similar to many spectral fitting approaches for nonlinear functions. Since we haven't gone through this, we will go through Bishop's motivating example, that of a local quadratic approximation.

Let's first do a Taylor expansion of $E(\mathbf{w})$ around some point ${\mathbf{\hat w}}$

$$E(\mathbf{w}) \simeq E(\hat{\mathbf{w}}) + (\mathbf{w} - \hat{\mathbf{w}})^T \mathbf{b} + \frac{1}{2} (\mathbf{w}-\hat{\mathbf{w}})^T \mathbf{H} (\mathbf{w} - \hat{\mathbf{w}})$$

keeping only terms through the quadratic. 

Here, $\mathbf{b}$ is the gradient of $E$ evaluated at $\hat{\mathbf{w}}$

$$\mathbf{b} \equiv \nabla E |_{\mathbf{w}=\hat{\mathbf{w}}}$$

and the Hessian matrix $\mathbf{H} = \nabla \nabla E$ has elements

$$(\mathbf{H})_{ij} = \frac{\partial E}{\partial w_i \partial w_j} \lvert _{\mathbf{w}=\hat{\mathbf{w}}}.$$

Solving the equation above for $\nabla E$, we find that the local approximation to the gradient is

$$\nabla E \simeq \mathbf{b} + \mathbf{H}(\mathbf{w} - \hat{\mathbf{w}}).$$

Next consider $\mathbf{w}^\star$ that is local minimum of the error function. Because $\nabla E = 0$ at $\mathbf{w}^\star$, then

$$E(\mathbf{w}) \simeq E(\mathbf{w}^\star) + \frac{1}{2} (\mathbf{w} -
\mathbf{w}^\star)^T \mathbf{H} (\mathbf{w} - \mathbf{w}^\star)$$

where the Hessian matrix has been evaluated at $\mathbf{w}^\star$.

It may be helpful to interpret this result geometrically. Consider the eigenvalue equation for the Hessian matrix:

$$\mathbf{H} \mathbf{u}_i = \lambda_i \mathbf{u}_i$$

where of course $\mathbf{u}_i$ are the eigenvectors and $\lambda_i$ are the eigenvalues. Since the eigenvectors form an orthonormal set, we can expand $(\mathbf{w}-\mathbf{w}^\star)$ as a linear function of eigenvectors as

$$\mathbf{w}-\mathbf{w}^\star = \sum_i \alpha_i \mathbf{u}_i.$$

This is of course a change of variables to the eigenvectors $\mathbf{u}$, where the origin is at $\mathbf{w}^\star$ and the axes are rotated to alighn with the eigenvectors.

Now, the error function can be written in the form

$$E(\mathbf{w}) = E(\mathbf{w}^\star) + \frac{1}{2} \sum_i \lambda_i
\alpha_i^2.$$

A matrix is positive definite if and only if

$$\mathbf{v}^T \mathbf{H} \mathbf{v} > 0$$

for all $\mathbf{v}$. Because the eigenvectors form a complete set and span the space, an arbitrary vector $\mathbf{v}$ can be written as

$$\mathbf{v}= \sum_i c_i \mathbf{u}_i.$$

So,

$$\mathbf{v}^T \mathbf{H} \mathbf{v} = \sum_i c_i^2 \lambda_i.$$

Therefore, $\mathbf{H}$ will be positive definite if all of its eigenvalues are positive. So in the new coordinate system, whose basis vectors are given by the eigenvectors, the contours of constant $E$ are ellipses centered on the origin. 

In a one-dimensional weight space, a stationary point $w^\star$ will be a minimum if

$$\frac{\partial^2 E}{\partial w^2} |_{w^\star} > 0.$$

So in $D$-dimensions, the Hessian matrix, evaluated at $\mathbf{w}^\star$ should be positive definite.

Bishop goes on to discuss the fact that the use of the gradient information saves computational steps. He also discusses the fact that the update

$$\mathbf{w}^{(\tau+1)} = \mathbf{w}^{(\tau)} + \eta \nabla E(\mathbf{w}^{(\tau)}) $$

can be done all at once (gradient descent), for the whole data set, or sequentially, (sequential gradient descent).



### Error Backpropagation

Now that we have established that the gradient will be useful for determining the parameters of the neural net, we need to discuss exactly how to get them. It turns out that a computationally efficient method involves a scheme in which the information is alternately sent forward and backward through the network. This method is called _backpropagation_.

The method involves two stages. The first stage involves evaluation of the derivatives with respect to the weights. It is in this step that the backpropagation occurs. The second step involves using the derivatives to compute adjustments to the weights.

#### Evaluation of Error-function Derivatives

Consider a general error function of the following form:

$$E(\mathbf{w}) = \sum_{n=1}^N E_n(\mathbf{w}), $$

i.e., the error function is a sum of error functions for each datapoint in the training set. So we will first consider the problem of evalutating one $\nabla E_n(\mathbf{w})$ for one term in the error function.

Consider first a linear model in which the outputs $y_k$ are linear combinations of the input variables $x_i$

$$y_k = \sum_i w_{ki} x_i$$

and an error function of the form

$$E_n = \frac{1}{2} \sum_k (y_{nk} - t_{nk} )^2$$

where $y_{nk} = y_k(\mathbf{x}_n, \mathbf{w})$. The gradient of this error function with respect to $w_{ji}$ is

$$\frac{\partial E_n}{\partial w_{ji}} = (y_{nj} - t_{nj}) x_{ni}.$$

This can be interpreted as a product of an "error signal" $y_{nj} - t_{nj}$ (i.e., difference between observation in the training set and model to be derived, at the output end of the link $w_{ji}$) and the variable $x_{ni}$ (associated with the input end of the link). Also note the similarity with formulas that arose using the logistic sigmoid activation function and the associated cross-entropy error function.

For the general feed-forward network (i.e., not the two layer but the more general network), each unit computes a weighted sum of its inputs of the form:

$$a_j = \sum_i w_{ji} z_i,$$

where $z_i$ is the activation of a unit, or an input, but at any rate, it sends the connection to unit $j$ and $w_{ji}$ is the weight associated with that connection. The sum in this equation can also
be transformed by the nonlinear activation function $h$ to give

$$z_j= h(a_j).$$

Consider that we have computed the forward propagation through the network using the current values of the parameters according to these two equations.

Next consider evaluation of the derivative of $E_n$ with respect to weight $w_{ji}$. Note that the weights and everything will be specific to $n$, but that dependence will be implicit by the lack of the boldface. We use the chain rule to obtain the derivative:

$$\frac{\partial E_n}{\partial w_{ji}} = \frac{\partial E_n}{\partial a_j} \frac{\partial a_j}{\partial w_{ji}}.$$

Define $\delta_j$, referred to as the errors, as

$$\delta_j \equiv \frac{\partial E_n}{\partial a_j}.$$

Using the equation above for $a_j$, we see

$$\frac{\partial a_j}{\partial w_{ji}} = z_i.$$

Substituting these two relationships into the chain rule result yields

$$\frac{\partial E_n}{\partial w_{ij}} = \delta_j z_i.$$

This is a fair simplification of the gradient: it is seen to be the product of the $\delta$ at the output end of the weight by the value of $z$ for the unit at the input end of the weight. So in order to evaluate the derivatives, we only need to compute the value of $\delta_j$ for each hidden and output unit, and then apply the equation above.

Now we need to deal with additional dependence of the connections between the hidden units and the outputs.  For the output, we have

$$\delta_k = y_k - t_k.$$

So we can again make use of the chain rule

$$\delta_j \equiv \frac{\partial E_n}{\partial a_j} = \sum_k \frac{\partial E_n}{\partial a_k} \frac{\partial a_k}{\partial a_j}$$

where the sum is running over $k$, which indexes the output. The arrangement is shown in Bishop Figure 5.7, reproduced below.

![sketch of Bishop figure 5.7](http://www.nhn.ou.edu/~leighly/bishop_5.7_sketch.tiff)

#### A Simple Example

The procedure above will become a little bit clearer when we consider a specific example - that of a two-layer neural network.  This is a commonly used form.  

We will also use a simple sum-of- squares error, in which the output units have linear activation functions so that $y_k = a_k$.  (It  is my impression that this type of activation function is appropriate when the output is a  regression, while a sigmoidal activation function would be appropriate if the output is a classification.  Well, that is what they say in Bishop, but it doesn't seem to be always the case.)  

The hidden units have sigmoidal activation functions:

$$h(a) \equiv \tanh(a)$$

where

$$\tanh(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}}.$$

It is noteworthy that this activiation function has a nice simple derivative:

$$h^\prime (a) = 1-h(a)^2.$$

We will also use a standard sum-of-squares error function, so that for pattern (i.e., input data point) $n$, the error is given by

$$E_n = \frac{1}{2} \sum_{k=1}^K (y_k - t_k)^2$$

where $y_k$ is the activation of output unit $k$ and $t_k$ is the corresponding target.

First perform the forward propagation for each pattern in the data set, as follows:

$$a_j = \sum_{i=0}^D w_{ji}^{(1)} x_i$$

$$z_j = \tanh(a_j)$$

$$y_k = \sum_{j=0}^M w_{kj}^{(2)} z_j.$$

Next, compute the $\delta$s for each output unit using:

$$\delta_k = y_k - t_k.$$

Then backpropagate to find the $\delta$s for the hidden uints using

$$\delta_j=(1-z_j^2) \sum_{k=1}^K w_{kj} \delta_k.$$

Finally, the derivatives with respect to the first-layer and second-layer weights are given by

$$\frac{\partial E_n}{\partial w_{ji}^{(1)}} = \delta_j x_i,$$

$$\frac{\partial E_n}{\partial w_{kj}^{(2)}} = \delta_k z_j.$$

These are then used to update the $w$s.

Neural net (technically, multi-layer perceptron) classifier is implemented in scikit-learn as [MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).  

Let's try to train on a generated sample and see what kind of decision boundary we get.


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.datasets import make_moons, make_circles, make_classification
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)



In [ ]:
plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')



In [ ]:
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)


In [ ]:
plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')



It is important to pre-process or standardize your data set.  You should use the same transformation for the training and the test set.  This can be implemented conveniently using scikit-learn class [StandardScalar](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) removes the mean and scales to unit variance.

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)


In [ ]:
plt.plot(X[y==0,0],X[y==0,1],'o')
plt.plot(X[y==1,0],X[y==1,1],'o')



There is also a convenient way to split into training and test sets using [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)


In [ ]:
plt.plot(X_train[y_train==0,0],X_train[y_train==0,1],'o')
plt.plot(X_train[y_train==1,0],X_train[y_train==1,1],'o')
plt.plot(X_test[y_test==0,0],X_test[y_test==0,1],'o')
plt.plot(X_test[y_test==1,0],X_test[y_test==1,1],'o')




In [ ]:
clf_mlp=MLPClassifier(alpha=1)

clf_mlp.fit(X_train,y_train)
score=clf_mlp.score(X_test,y_test)

In [ ]:
h=0.02
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

Z_mlp = clf_mlp.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
Z_mlp=Z_mlp.reshape(xx.shape)

In [ ]:
plt.plot(X_train[y_train==0,0],X_train[y_train==0,1],'o')
plt.plot(X_train[y_train==1,0],X_train[y_train==1,1],'o')
plt.plot(X_test[y_test==0,0],X_test[y_test==0,1],'o')
plt.plot(X_test[y_test==1,0],X_test[y_test==1,1],'o')

plt.contour(xx,yy,Z_mlp,[0.5],colors='k')

